1. match backend to fsp
2. match fsp to backend

exact matches for description number only  = 9999 / 10630 = 93.7%

# Data insights
fsp has records that are near identical where the only difference is the reporting tag which is 'bills and payments' and 'fees and commissions' where the records tagged 'bills and payments' have filled 'backend_accounts' and 'backend_transaction_ids' while those tagged with 'fees and commissions' lack these fields.

The bills and payments(client paying) and fees and commissions(fsp charge) have the same description number and date. 
Difference is the reporting tag and amount.

* for non-numeric descriptions, can you extract description number from original reference present in metadata in fsp ??




In [1]:
import  pandas as pd
import recordlinkage
import re

nomba_gtb_fsp = pd.read_csv('Nomba_GTB_BILLS - fsp_16_mar_2024_31_mar_2024.csv', index_col='id_fsp')
nomba_gtb_backend = pd.read_csv('Nomba_GTB_BILLS - Backend_1_Mar_2024_31_2024.csv', index_col='id_backend')

#nomba_gtb_backend

,transaction_date_backend,value_date,transaction_type,amount_backend,amount_currency,balance,balance_currency,matching_reference,reporting_tag,is_reversal,...,updated,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description
id_backend,,,,,,,,,,,,,,,,,,,,,
684754662,2024-03-31,2024-03-31 21:21:30.000000 UTC,C,2950,NGN,0,NGN,[3@39188343199314622950.00],transfers,False,...,2024-04-12 01:52:53.248606 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 21:21:30, origina...",[transfers],3918834319931462/SUCCESS
684754663,2024-03-31,2024-03-31 20:20:30.000000 UTC,C,350,NGN,0,NGN,[3@2347647791697967350.00],transfers,False,...,2024-04-12 01:52:53.248606 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 20:20:30, origina...",[transfers],2347647791697967/SUCCESS
684754665,2024-03-31,2024-03-31 23:14:38.000000 UTC,C,1950,NGN,0,NGN,[3@38925732955313971950.00],transfers,False,...,2024-04-12 01:52:53.248606 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 23:14:38, origina...",[transfers],3892573295531397/SUCCESS
684754666,2024-03-31,2024-03-31 21:03:38.000000 UTC,C,1700,NGN,0,NGN,[3@72433543269619881700.00],transfers,False,...,2024-04-12 01:52:53.248606 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 21:03:38, origina...",[transfers],7243354326961988/SUCCESS
684754668,2024-03-31,2024-03-31 22:00:07.000000 UTC,C,3950,NGN,0,NGN,[3@14616371572939783950.00],transfers,False,...,2024-04-12 01:52:53.248606 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 22:00:07, origina...",[transfers],1461637157293978/SUCCESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684756326,2024-03-03,2024-03-03 18:14:09.000000 UTC,C,18000,NGN,0,NGN,[3@857975845556732518000.00],transfers,False,...,2024-05-24 00:17:49.000000 UTC,[70],[489667289],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 18:14:09, origina...",[transfers],8579758455567325/SUCCESS
684758495,2024-03-03,2024-03-03 16:04:26.000000 UTC,C,500,NGN,0,NGN,[3@2921958749136149500.00],transfers,False,...,2024-05-24 00:17:49.000000 UTC,[70],[489667203],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 16:04:26, origina...",[transfers],2921958749136149/SUCCESS
684756787,2024-03-05,2024-03-05 10:52:03.000000 UTC,C,200,NGN,0,NGN,[3@1638195999657187200.00],transfers,False,...,2024-05-24 00:17:49.000000 UTC,[70],[490523094],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-05 10:52:03, origina...",[transfers],1638195999657187/SUCCESS


nomba_gtb_backend.columns

###Preprocessing 

    1. Ensuring col names in the data sources are different. ( id, transaction date, amount, extracted description) 
    2. Ensuring transaction date is in the same format(no time included).
    3. Extracting description number from both backend and frontend.
    4. Amount in the same format (2d).

In [2]:
nomba_gtb_backend['amount_backend'] = nomba_gtb_backend['amount_backend'].astype(float).map('{:.2f}'.format)
nomba_gtb_fsp['amount_fsp'] = nomba_gtb_fsp['amount_fsp'].astype(float).map('{:.2f}'.format)

In [3]:
nomba_gtb_backend['description_number'] = nomba_gtb_backend['description'].str.split('/').str[0]
#nomba_gtb_backend

In [4]:
def extract_fsp_description_number(text):
    match = re.search(r'TP-COSMIC\s+(\d+)', text)
    return match.group(1) if match else None


nomba_gtb_fsp['description_number_fsp'] = nomba_gtb_fsp['description'].apply(extract_fsp_description_number)
#nomba_gtb_fsp

In [5]:
# Filter for 'bills_payments'
nomba_gtb_fsp_bills_payments = nomba_gtb_fsp[nomba_gtb_fsp['reporting_tag'] == 'bill_payments']

# Filter for 'fees_and_commissions'
nomba_gtb_fsp_fees_and_commissions = nomba_gtb_fsp[nomba_gtb_fsp['reporting_tag'] == 'fees_and_commissions']



nomba_gtb_fsp_fees_and_commissions

nomba_gtb_fsp_bills_payments

Work with bills and payments first

Creating indexer object.
Either blocking or full index.

Individual indexer object for comparing fsp to backend and another for backend to fsp

In [6]:
# FSP TO BACKEND
indexer = recordlinkage.Index()
indexer.block(left_on='transaction_date_fsp',right_on='transaction_date_backend')

# BACKEND TO FSP
indexer_backend = recordlinkage.Index()
indexer_backend.block(left_on='transaction_date_backend',right_on='transaction_date_fsp')

indexer_backend_fees = recordlinkage.Index()
indexer_backend_fees.block(left_on='transaction_date_backend',right_on='transaction_date_fsp')



<Index>

Blocking on transaction date means only transactions made on the same date are compared.
Incase a transaction date was erroneously recorded on either data source, a false positive / negative is possible

nomba_gtb_fsp

In [7]:
# CANDIDATES FOR FSP TO BACKEND
candidates_bills = indexer.index(nomba_gtb_fsp_bills_payments,nomba_gtb_backend)
candidates_fees = indexer.index(nomba_gtb_fsp_fees_and_commissions, nomba_gtb_backend)
candidates_combined_fsp = indexer.index(nomba_gtb_fsp, nomba_gtb_backend)
len(candidates_combined_fsp)
#len(candidates_bills)

# CANDIDATES FOR BACKEND TO FSP bills
candidates_backend_bills = indexer_backend.index(nomba_gtb_backend, nomba_gtb_fsp_bills_payments)
len(candidates_backend_bills)

# CANDIDATES FOR BACKEND TO FSP FEES
candidates_backend_fees = indexer_backend.index(nomba_gtb_backend, nomba_gtb_fsp_fees_and_commissions)
len(candidates_backend_fees)

# full index produces 11m+ candidates

1938315

compare options: exact, string, date , add.

Set threshold and method.

### Comparison methods
    1. Jaro-winkler
    2. Levenshtein
    3. Cosine Similarity
    4. Damerau-Levenshtein
    5. Hamming distance
    6. Soundex - not relevant here
    
Ref : https://www.getcensus.com/blog/implementing-entity-resolution-with-python-record-linkage

In [9]:
# initialize Compare classes
compare_bills = recordlinkage.Compare()
compare_fees = recordlinkage.Compare()
compare_combined_fsp = recordlinkage.Compare()
compare_backend_bills = recordlinkage.Compare()
compare_backend_fees = recordlinkage.Compare()


compare_bills.exact("description_number_fsp","description_number" ,label = 'desc_number')
compare_bills.string("amount_fsp","amount_backend", method='jarowinkler', label = 'amount')

compare_fees.exact("description_number_fsp","description_number" ,label = 'desc_number')
compare_fees.string("amount_fsp","amount_backend", method='jarowinkler', label = 'amount')
#compare.string("reporting_tag","reporting_tag", method='jarowinkler', label = 'reporting_tag_combined')

compare_combined_fsp.exact("description_number_fsp","description_number" ,label = 'desc_number')
compare_combined_fsp.string("amount_fsp","amount_backend", method='jarowinkler', label = 'amount')

compare_backend_bills.exact("description_number","description_number_fsp" ,label = 'desc_number')
compare_backend_bills.string("amount_backend","amount_fsp", method='jarowinkler', label = 'amount')

compare_backend_fees.exact("description_number","description_number_fsp" ,label = 'desc_number')
compare_backend_fees.string("amount_backend","amount_fsp", method='jarowinkler', label = 'amount')


    
#compare.string("transaction_date_fsp","transaction_date_backend", label ='transaction_date')
features_bills = compare_bills.compute(candidates_bills, nomba_gtb_fsp_bills_payments, nomba_gtb_backend)
features_fees = compare_fees.compute(candidates_fees, nomba_gtb_fsp_fees_and_commissions, nomba_gtb_backend)
features_combined_fsp = compare_combined_fsp.compute(candidates_combined_fsp,nomba_gtb_fsp,nomba_gtb_backend)

# backend fsp bills
features_backend_bills = compare_backend_bills.compute(candidates_backend_bills,nomba_gtb_backend,nomba_gtb_fsp_bills_payments)

# backend fsp fees
#features_backend_fees = compare_backend_fees.compute(candidates_backend_fees,nomba_gtb_backend,nomba_gtb_fsp_fees_and_commissions)


When a threshold is defined, binary values are returned
Without a threshold, a similarity score is defined without classifying pairs

features_bills

features_fees

features_combined_fsp

In [10]:
features_backend_bills

desc_number    amount
id_backend id_fsp                          
684754662  493532329            0  0.849206
           493532155            0  0.630952
           493532407            0  0.864286
           493532377            0  0.630952
           493531853            0  0.630952
...                           ...       ...
684762638  493528081            0  0.662698
           493528051            0  0.782540
           493528427            0  0.904762
           493528251            0  0.785714
           493528175            0  0.782540

[1936834 rows x 2 columns]

In [11]:
features_bills['sum']= features_bills['desc_number'] + features_bills['amount']
features_fees['sum']= features_fees['desc_number'] + features_fees['amount']
features_combined_fsp['sum']= features_combined_fsp['desc_number'] + features_combined_fsp['amount']
features_backend_bills['sum']= features_backend_bills['desc_number'] + features_backend_bills['amount']
#features_backend_fees['sum']= features_backend_fees['desc_number'] + features_backend_fees['amount']

#features_combined_fsp

In [12]:

best_match_fsp_bills = features_bills.loc[features_bills.groupby('id_fsp')['sum'].idxmax()]
best_match_fsp_fees = features_fees.loc[features_fees.groupby('id_fsp')['sum'].idxmax()]
best_match_fsp_combined = features_combined_fsp.loc[features_combined_fsp.groupby('id_fsp')['sum'].idxmax()]
best_match_backend_bills = features_backend_bills.loc[features_backend_bills.groupby('id_backend')['sum'].idxmax()]
#best_match_backend_fees = features_backend_fees.loc[features_backend_fees.groupby('id_backend')['sum'].idxmax()]

#best_match_fsp_fees

best_match_fsp_bills

best_match_fsp_combined

In [13]:
best_match_backend_bills

,,desc_number,amount,sum
id_backend,id_fsp,,,
684754594,493531435,1,1.000000,2.000000
684754595,493531043,1,1.000000,2.000000
684754596,493531371,1,1.000000,2.000000
684754597,493530791,1,1.000000,2.000000
684754598,493531319,1,1.000000,2.000000
...,...,...,...,...
684765329,493524299,0,0.957143,0.957143
684765341,493523671,1,1.000000,2.000000
684765342,493523005,1,1.000000,2.000000


## ELIMINATE USED ID_BACKEND

best_match_backend_bills

In [14]:
#best_match_backend_bills.reset_index()
merged_df = pd.merge(nomba_gtb_backend, best_match_backend_bills, 
                     left_on='id_backend', right_on='id_backend', how='left', indicator=True)

merged_df

,transaction_date_backend,value_date,transaction_type,amount_backend,amount_currency,balance,balance_currency,matching_reference,reporting_tag,is_reversal,...,other_accounts,other_transaction_ids,metadata,tags,description,description_number,desc_number,amount,sum,_merge
id_backend,,,,,,,,,,,,,,,,,,,,,
684754662,2024-03-31,2024-03-31 21:21:30.000000 UTC,C,2950.00,NGN,0,NGN,[3@39188343199314622950.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-31 21:21:30, origina...",[transfers],3918834319931462/SUCCESS,3918834319931462,0.0,0.952381,0.952381,both
684754663,2024-03-31,2024-03-31 20:20:30.000000 UTC,C,350.00,NGN,0,NGN,[3@2347647791697967350.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-31 20:20:30, origina...",[transfers],2347647791697967/SUCCESS,2347647791697967,0.0,1.000000,1.000000,both
684754665,2024-03-31,2024-03-31 23:14:38.000000 UTC,C,1950.00,NGN,0,NGN,[3@38925732955313971950.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-31 23:14:38, origina...",[transfers],3892573295531397/SUCCESS,3892573295531397,0.0,1.000000,1.000000,both
684754666,2024-03-31,2024-03-31 21:03:38.000000 UTC,C,1700.00,NGN,0,NGN,[3@72433543269619881700.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-31 21:03:38, origina...",[transfers],7243354326961988/SUCCESS,7243354326961988,0.0,0.957143,0.957143,both
684754668,2024-03-31,2024-03-31 22:00:07.000000 UTC,C,3950.00,NGN,0,NGN,[3@14616371572939783950.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-31 22:00:07, origina...",[transfers],1461637157293978/SUCCESS,1461637157293978,0.0,0.957143,0.957143,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684756326,2024-03-03,2024-03-03 18:14:09.000000 UTC,C,18000.00,NGN,0,NGN,[3@857975845556732518000.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-03 18:14:09, origina...",[transfers],8579758455567325/SUCCESS,8579758455567325,NaN,NaN,NaN,left_only
684758495,2024-03-03,2024-03-03 16:04:26.000000 UTC,C,500.00,NGN,0,NGN,[3@2921958749136149500.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-03 16:04:26, origina...",[transfers],2921958749136149/SUCCESS,2921958749136149,NaN,NaN,NaN,left_only
684756787,2024-03-05,2024-03-05 10:52:03.000000 UTC,C,200.00,NGN,0,NGN,[3@1638195999657187200.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-05 10:52:03, origina...",[transfers],1638195999657187/SUCCESS,1638195999657187,NaN,NaN,NaN,left_only


In [15]:
# Filter out rows that have a match (i.e., rows present in both DataFrames)
nomba_gtb_backend_fees = merged_df[merged_df['_merge'] == 'left_only']

nomba_gtb_backend_fees

,transaction_date_backend,value_date,transaction_type,amount_backend,amount_currency,balance,balance_currency,matching_reference,reporting_tag,is_reversal,...,other_accounts,other_transaction_ids,metadata,tags,description,description_number,desc_number,amount,sum,_merge
id_backend,,,,,,,,,,,,,,,,,,,,,
684758450,2024-03-01,2024-03-01 10:32:44.000000 UTC,C,500.00,NGN,0,NGN,[3@65E1AECD99B90D00193F4F8B500.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-01 10:32:44, origina...",[transfers],65e1aecd99b90d00193f4f8b/SUCCESS,65e1aecd99b90d00193f4f8b,NaN,NaN,NaN,left_only
684758833,2024-03-02,2024-03-02 20:32:20.000000 UTC,C,500.00,NGN,0,NGN,[3@65E38CD52F50420019C0813A500.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-02 20:32:20, origina...",[transfers],65e38cd52f50420019c0813a/SUCCESS,65e38cd52f50420019c0813a,NaN,NaN,NaN,left_only
684759440,2024-03-13,2024-03-13 11:00:07.000000 UTC,C,1000.00,NGN,0,NGN,NaN,others,False,...,NaN,NaN,"{transaction_date=2024-03-13 11:00:07, origina...","[others, unknown_pattern]",nan/SUCCESS,nan,NaN,NaN,NaN,left_only
684759701,2024-03-01,2024-03-01 17:48:31.000000 UTC,C,1000.00,NGN,0,NGN,[3@65E214EF33BAE800124531651000.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-01 17:48:31, origina...",[transfers],65e214ef33bae80012453165/SUCCESS,65e214ef33bae80012453165,NaN,NaN,NaN,left_only
684764443,2024-03-08,2024-03-08 21:49:03.000000 UTC,C,9000.00,NGN,0,NGN,[3@65EB87CF2955AC00195A2AB99000.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-08 21:49:03, origina...",[transfers],65eb87cf2955ac00195a2ab9/SUCCESS,65eb87cf2955ac00195a2ab9,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684756326,2024-03-03,2024-03-03 18:14:09.000000 UTC,C,18000.00,NGN,0,NGN,[3@857975845556732518000.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-03 18:14:09, origina...",[transfers],8579758455567325/SUCCESS,8579758455567325,NaN,NaN,NaN,left_only
684758495,2024-03-03,2024-03-03 16:04:26.000000 UTC,C,500.00,NGN,0,NGN,[3@2921958749136149500.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-03 16:04:26, origina...",[transfers],2921958749136149/SUCCESS,2921958749136149,NaN,NaN,NaN,left_only
684756787,2024-03-05,2024-03-05 10:52:03.000000 UTC,C,200.00,NGN,0,NGN,[3@1638195999657187200.00],transfers,False,...,NaN,NaN,"{transaction_date=2024-03-05 10:52:03, origina...",[transfers],1638195999657187/SUCCESS,1638195999657187,NaN,NaN,NaN,left_only


In [16]:
# Drop the additional columns added by the merge
nomba_gtb_backend_fees = nomba_gtb_backend_fees.drop(columns=['desc_number', 'amount', 'sum', '_merge'])
nomba_gtb_backend_fees

,transaction_date_backend,value_date,transaction_type,amount_backend,amount_currency,balance,balance_currency,matching_reference,reporting_tag,is_reversal,...,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description,description_number
id_backend,,,,,,,,,,,,,,,,,,,,,
684758450,2024-03-01,2024-03-01 10:32:44.000000 UTC,C,500.00,NGN,0,NGN,[3@65E1AECD99B90D00193F4F8B500.00],transfers,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-01 10:32:44, origina...",[transfers],65e1aecd99b90d00193f4f8b/SUCCESS,65e1aecd99b90d00193f4f8b
684758833,2024-03-02,2024-03-02 20:32:20.000000 UTC,C,500.00,NGN,0,NGN,[3@65E38CD52F50420019C0813A500.00],transfers,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-02 20:32:20, origina...",[transfers],65e38cd52f50420019c0813a/SUCCESS,65e38cd52f50420019c0813a
684759440,2024-03-13,2024-03-13 11:00:07.000000 UTC,C,1000.00,NGN,0,NGN,NaN,others,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-13 11:00:07, origina...","[others, unknown_pattern]",nan/SUCCESS,nan
684759701,2024-03-01,2024-03-01 17:48:31.000000 UTC,C,1000.00,NGN,0,NGN,[3@65E214EF33BAE800124531651000.00],transfers,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-01 17:48:31, origina...",[transfers],65e214ef33bae80012453165/SUCCESS,65e214ef33bae80012453165
684764443,2024-03-08,2024-03-08 21:49:03.000000 UTC,C,9000.00,NGN,0,NGN,[3@65EB87CF2955AC00195A2AB99000.00],transfers,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-08 21:49:03, origina...",[transfers],65eb87cf2955ac00195a2ab9/SUCCESS,65eb87cf2955ac00195a2ab9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684756326,2024-03-03,2024-03-03 18:14:09.000000 UTC,C,18000.00,NGN,0,NGN,[3@857975845556732518000.00],transfers,False,...,[70],[489667289],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 18:14:09, origina...",[transfers],8579758455567325/SUCCESS,8579758455567325
684758495,2024-03-03,2024-03-03 16:04:26.000000 UTC,C,500.00,NGN,0,NGN,[3@2921958749136149500.00],transfers,False,...,[70],[489667203],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 16:04:26, origina...",[transfers],2921958749136149/SUCCESS,2921958749136149
684756787,2024-03-05,2024-03-05 10:52:03.000000 UTC,C,200.00,NGN,0,NGN,[3@1638195999657187200.00],transfers,False,...,[70],[490523094],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-05 10:52:03, origina...",[transfers],1638195999657187/SUCCESS,1638195999657187


## Match backend_bills first then backend_fees. 
nomba_gtb_backend_fees contains id_backend that are to match to reporting tag fees_and_commissions in fsp

In [17]:
backend_fees = nomba_gtb_backend_fees
backend_fees

,transaction_date_backend,value_date,transaction_type,amount_backend,amount_currency,balance,balance_currency,matching_reference,reporting_tag,is_reversal,...,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description,description_number
id_backend,,,,,,,,,,,,,,,,,,,,,
684758450,2024-03-01,2024-03-01 10:32:44.000000 UTC,C,500.00,NGN,0,NGN,[3@65E1AECD99B90D00193F4F8B500.00],transfers,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-01 10:32:44, origina...",[transfers],65e1aecd99b90d00193f4f8b/SUCCESS,65e1aecd99b90d00193f4f8b
684758833,2024-03-02,2024-03-02 20:32:20.000000 UTC,C,500.00,NGN,0,NGN,[3@65E38CD52F50420019C0813A500.00],transfers,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-02 20:32:20, origina...",[transfers],65e38cd52f50420019c0813a/SUCCESS,65e38cd52f50420019c0813a
684759440,2024-03-13,2024-03-13 11:00:07.000000 UTC,C,1000.00,NGN,0,NGN,NaN,others,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-13 11:00:07, origina...","[others, unknown_pattern]",nan/SUCCESS,nan
684759701,2024-03-01,2024-03-01 17:48:31.000000 UTC,C,1000.00,NGN,0,NGN,[3@65E214EF33BAE800124531651000.00],transfers,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-01 17:48:31, origina...",[transfers],65e214ef33bae80012453165/SUCCESS,65e214ef33bae80012453165
684764443,2024-03-08,2024-03-08 21:49:03.000000 UTC,C,9000.00,NGN,0,NGN,[3@65EB87CF2955AC00195A2AB99000.00],transfers,False,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-08 21:49:03, origina...",[transfers],65eb87cf2955ac00195a2ab9/SUCCESS,65eb87cf2955ac00195a2ab9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684756326,2024-03-03,2024-03-03 18:14:09.000000 UTC,C,18000.00,NGN,0,NGN,[3@857975845556732518000.00],transfers,False,...,[70],[489667289],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 18:14:09, origina...",[transfers],8579758455567325/SUCCESS,8579758455567325
684758495,2024-03-03,2024-03-03 16:04:26.000000 UTC,C,500.00,NGN,0,NGN,[3@2921958749136149500.00],transfers,False,...,[70],[489667203],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 16:04:26, origina...",[transfers],2921958749136149/SUCCESS,2921958749136149
684756787,2024-03-05,2024-03-05 10:52:03.000000 UTC,C,200.00,NGN,0,NGN,[3@1638195999657187200.00],transfers,False,...,[70],[490523094],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-05 10:52:03, origina...",[transfers],1638195999657187/SUCCESS,1638195999657187


nomba_gtb_fsp

In [18]:
# CANDIDATES FOR BACKEND TO FSP FEES
candidates_backend_fees_post = indexer_backend_fees.index(nombackend_fees, nomba_gtb_fsp)
len(candidates_backend_fees_post)

0

In [21]:
best_match_backend_bills = features_bills.loc[features_bills.groupby('id_backend')['sum'].idxmax()]
best_match_backend_fees = features_fees.loc[features_fees.groupby('id_backend')['sum'].idxmax()]
best_match_backend_fees

,,desc_number,amount,sum
id_fsp,id_backend,,,
493531436,684754594,1,0.683333,1.683333
493531044,684754595,1,0.565079,1.565079
493531372,684754596,1,0.547222,1.547222
493530792,684754597,1,0.683333,1.683333
493531320,684754598,1,0.736429,1.736429
...,...,...,...,...
493524260,684765329,0,0.565079,0.565079
493523672,684765341,1,0.565079,1.565079
493523006,684765342,1,0.565079,1.565079


In [22]:
# Have to redo to match backend to fsp where on the left is backend and right_on is fsp

backend_best_match_combined = features_combined_fsp.loc[features_combined_fsp.groupby('id_backend')['sum'].idxmax()]
backend_best_match_combined

,,desc_number,amount,sum
id_fsp,id_backend,,,
493531435,684754594,1,1.000000,2.000000
493531043,684754595,1,1.000000,2.000000
493531371,684754596,1,1.000000,2.000000
493530791,684754597,1,1.000000,2.000000
493531319,684754598,1,1.000000,2.000000
...,...,...,...,...
493524299,684765329,0,0.957143,0.957143
493523671,684765341,1,1.000000,2.000000
493523005,684765342,1,1.000000,2.000000


best_match['sum'].value_counts().sort_index(ascending=False)

potential_matches = features[features.sum(axis=1) >= 1.5].reset_index().sort_index(ascending=False)
potential_matches

exact matches for description number only  = 9999 / 10630 = 93%
